In [2]:
import pandas as pd
# import dask.dataframe as dd
# import dask.array as da
import anndata as ad
import pandas as pd
from scipy.sparse import csr_matrix, vstack, hstack
import numpy as np
from pathlib import Path
from scipy.io import mmwrite


# File paths
obs_file_path = "cell_barcde_labels.csv"
count_matrix_file_path = "brain_atlas_full_counts_table.csv"

# Paths
obs_file_path = "./data/cell_barcode_labels.csv"
count_matrix_file_path = "./data/brain_atlas_full_counts_table.csv"
h5ad_file_path = "./data/dask_output_data.h5ad"



In [ ]:


# Specify a larger sample size (e.g., 1e7 bytes)
sample_size = 1e7


cells = pd.read_csv(obs_file_path,index_col=0)


# Specify data types
# Assuming that the first column is object (like string) and the rest are uint8.
dtypes = {0: 'object'}
for col in range(1, 713626):  # Adjust the range based on the number of columns
    dtypes[col] = 'uint8'

                              


In [16]:

# Define the chunk size
chunk_size = 10  # 

# Create an empty list to store sparse matrices from each chunk
chunk_counter = 0
sparse_matrices = []
genes = []
# 1. Read the CSV in chunks using a context manager
with pd.read_csv(count_matrix_file_path, header=0,dtype=dtypes, chunksize=chunk_size) as reader:
    for chunk in reader:

        print(f"chunk number {chunk_counter}")
        # 2. get the genes
        gene_chunk = chunk['genes']

        sparse_chunk = csr_matrix(chunk.iloc[:,1:].values.T)
        genes.append(gene_chunk)
        sparse_matrices.append(sparse_chunk)

        # after each 10 chunks write file and reset
        if (chunk_counter+1) % 10 == 0:
            sparse_matrix = hstack(sparse_matrices)
            genes_ = pd.DataFrame(index = pd.concat(genes, axis=0))
            genes_['chunk'] = chunk_counter

            # adata_dict = {}
            # adata_dict["X"] = sparse_matrix.transpose()
            # adata_dict["obs"] = cells
            # adata_dict["var"] = genes_
            # # adata_dict["dtype"] = np.float64
            # # adata_dict["obsm"] = dict(
            # #     a=da.random.random((M, 100)),
            # # )
            # # adata_dict["layers"] = dict(
            # #     a=da.random.random((M, N)),
            # # )
            print(f"matrix shape {sparse_matrix.shape} \n var shape {genes_.shape} obs shape {cells.shape[0]}")

 
            adata = ad.AnnData(X=sparse_matrix, obs=cells, var=genes_)
            # adata = ad.AnnData(**adata_dict)
            h5ad_file_path = f"./data/chunk2{chunk_counter+1:04d}_output_data.h5ad"
            adata.write_h5ad(h5ad_file_path)
            del adata
            print(f"wrote {genes_.shape} genes to adata file {h5ad_file_path}")
            sparse_matrices = []
            genes = []

        chunk_counter += 1



        



chunk number 0
chunk number 1
chunk number 2
chunk number 3
chunk number 4
chunk number 5
chunk number 6
chunk number 7
chunk number 8
chunk number 9
matrix shape (713626, 100) 
 var shape (100, 1) obs shape 713626
wrote (100, 1) genes to adata file ./data/chunk20009_output_data.h5ad
chunk number 10
chunk number 11
chunk number 12
chunk number 13
chunk number 14
chunk number 15
chunk number 16
chunk number 17
chunk number 18
chunk number 19
matrix shape (713626, 100) 
 var shape (100, 1) obs shape 713626
wrote (100, 1) genes to adata file ./data/chunk20019_output_data.h5ad
chunk number 20
chunk number 21


KeyboardInterrupt: 

In [15]:
chunkv = chunk.iloc[:,1:].values.T

In [6]:
adata = ad.AnnData(X=sparse_matrix.transpose(), obs=cells, var=genes_)


In [28]:
sparse_matrix = vstack(sparse_matrices)
genes_ = pd.DataFrame(index = pd.concat(genes, axis=0))

adata_dict = {}
adata_dict["X"] = sparse_matrix.transpose()
adata_dict["obs"] = cells
adata_dict["var"] = genes_

adata = ad.AnnData(**adata_dict)
h5ad_file_path = f"./data/chunk{chunk_counter+1:04d}_output_data.h5ad"
adata.write_h5ad(h5ad_file_path)

In [8]:
adata.var_names_make_unique()

In [11]:
adata.var_names

Index(['MIR1302-2HG', 'FAM138A', 'OR4F5', 'AL627309.1', 'AL627309.3',
       'AL627309.2', 'AL627309.5', 'AL627309.4', 'AP006222.2', 'AL732372.1',
       'OR4F29', 'AC114498.1', 'OR4F16', 'AL669831.2', 'LINC01409', 'FAM87B',
       'LINC01128', 'LINC00115', 'FAM41C', 'AL645608.6', 'AL645608.2',
       'AL645608.4', 'LINC02593', 'SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1',
       'PERM1', 'AL645608.7', 'HES4', 'ISG15', 'AL645608.1', 'AGRN',
       'AL645608.5', 'AL645608.8', 'RNF223', 'C1orf159', 'AL390719.3',
       'LINC01342', 'AL390719.2', 'TTLL10-AS1', 'TTLL10', 'TNFRSF18',
       'TNFRSF4', 'SDF4', 'B3GALT6', 'C1QTNF12', 'AL162741.1', 'UBE2J2',
       'LINC01786', 'SCNN1D', 'ACAP3', 'PUSL1', 'INTS11', 'AL139287.1', 'CPTP',
       'TAS1R3', 'DVL1', 'MXRA8', 'AURKAIP1', 'CCNL2', 'MRPL20-AS1', 'MRPL20',
       'AL391244.2', 'ANKRD65', 'AL391244.1', 'TMEM88B', 'LINC01770', 'VWA1',
       'ATAD3C', 'ATAD3B', 'ATAD3A', 'TMEM240', 'SSU72', 'AL645728.1',
       'FNDC10', 'AL691432.4', 'AL69143

In [13]:
adata.X

<713626x90 sparse matrix of type '<class 'numpy.uint8'>'
	with 2167712 stored elements in Compressed Sparse Column format>

In [25]:
            h5ad_file_path = f"./data/chunk{chunk_counter+1:04d}_output_data.h5ad"
            adata.write_h5ad(h5ad_file_path)

In [24]:
adata_dict["X"] = sparse_matrix.transpose()
genes_ = pd.DataFrame(index = pd.concat(genes, axis=0))
adata_dict["var"] = genes_


adata = ad.AnnData(**adata_dict)



In [20]:
adata_dict['X'].shape

(900, 713626)

In [16]:
adata_dict['obs'].shape

(713626, 3)

In [23]:
genes__ = pd.DataFrame(adata_dict['var'])

In [26]:
chunk_counter

9

In [ ]:
# Setup
from pathlib import Path

import h5py
from scipy import sparse

import anndata as ad
from anndata._core.sparse_dataset import SparseDataset
from anndata.experimental import read_elem, write_elem


def read_everything_but_X(pth) -> ad.AnnData:
    attrs = ["obs", "var", "obsm", "varm", "obsp", "varp", "uns"]
    with h5py.File(pth) as f:
        adata = ad.AnnData(**{k: read_elem(f[k]) for k in attrs})
    return adata


def concat_on_disk(input_pths: list[Path], output_pth: Path):
    """
    Params
    ------
    input_pths
        Paths to h5ad files which will be concatenated
    output_pth
        File to write as a result
    """
    annotations = ad.concat([read_everything_but_X(pth) for pth in input_pths])

    annotations.write_h5ad(output_pth)
    n_variables = annotations.shape[1]
    
    del annotations

    with h5py.File(out_pth, "a") as target:
        dummy_X = sparse.csr_matrix((0, n_variables), dtype="float32")
        dummy_X.indptr = dummy_X.indptr.astype("int64") # Guarding against overflow for very large datasets

        write_elem(target, "X", dummy_X)
        mtx = SparseDataset(target["X"])
        for p in pths:
            with h5py.File(p, "r") as src:
                mtx.append(SparseDataset(src["X"]))

In [ ]:
from pathlib import Path

import anndata as ad
import h5py
import pandas as pd
from scipy import sparse

tmp_dir = Path("tmp_h5ads/")
tmp_dir.mkdir(exist_ok=True)
backed_adatas = []

for prefix in list("abcde"):
    pth = tmp_dir / f"{prefix}.h5ad"
    adata = sc.AnnData(
        sparse.random(100, 100, density=0.3, format="csr"),
        obs=pd.DataFrame(index=[f"{prefix}-cell{i}" for i in range(100)])
    )
    adata.write(pth)
    backed_adatas.append(ad.read_h5ad(pth, backed="r"))

In [ ]:
with h5py.File("concatenated.h5ad", "w") as f:
    ad._io.h5ad.write_attribute(f, "X", sparse.csr_matrix((0, 100)))
    base_dset = ad._core.sparse_dataset.SparseDataset(f["X"])

    # Concatenate adatas
    for adata in backed_adatas:
        base_dset.append(adata.X)

    ad._io.h5ad.write_attribute(f, "obs", pd.concat((a.obs for a in backed_adatas)))
    ad._io.h5ad.write_attribute(f, "var", backed_adatas[0].var)

    # Thank you ivirshup for the solution. Just wanted to add, that if you are trying this with anndata version 0.9.1, 
    # you need to replace write_attribute with write_elem

From : https://github.com/scverse/anndata/issues/794

In [ ]:
with h5py.File(output_file, "w") as target:
    target.create_dataset(
        "X",
        (obs.shape[0], var.shape[0]),
         dtype="float32",
        chunks=(1000, 1000),
    )
    write_elem(target, "obs", obs)
    write_elem(target, "var", var)

# read in created output file
adata = ad.read(output_file, backed="r+")

counter = 0
for f in files:
    cur_X = ... # read in a subset of X
    adata[counter : counter + cur_X.shape[0], :].X = cur_X # store that subset of X
    counter += cur_X.shape[0]

In [ ]:

import h5py
import numpy as np
import pandas as pd
import anndata as ad
from anndata.experimental import write_elem

output_file="test.h5ad"
n_cols = 10
n_rows = 1000
obs = pd.DataFrame({"group":np.arange(n_rows)})
var = pd.DataFrame({"feature":np.arange(n_cols)})

# dummy function for chunked X
def get_X_chunk(chunksize=10):
    X = np.random.random((n_rows,n_cols))
    chunk_idx = 0
    for i in range(chunksize,len(X)+chunksize,chunksize):
        yield X[chunk_idx:min(i,len(X))]
        chunk_idx += chunksize
        
# initialize h5ad on file
with h5py.File(output_file, "w") as target:
    target.create_dataset(
        "X",
        (obs.shape[0], var.shape[0]),
         dtype="float32",
        chunks=(10, 10),
    )
    write_elem(target, "obs", obs)
    write_elem(target, "var", var)
    
# read in created output file
adata = ad.read(output_file, backed="r+")

# write chunks to file
counter = 0
for chunk in get_X_chunk():
    adata[counter : counter + chunk.shape[0], :].X = chunk # store that subset of X
    counter += chunk.shape[0]
   
### Error mode 1:

adata.raw = adata
# AttributeError: Could not find dataset for raw X in file: test.h5ad.

### Error mode 2:
print(adata.X)
# <HDF5 dataset "X": shape (1000, 10), type "<f4">

# Compare with:
print(AnnData(X=np.hstack([*get_X_chunk()])).X)
# -> array([[...]]